## 지역별 특보

In [5]:
import numpy as np
import requests
import pprint
import json
import pandas as pd
import datetime
from datetime import datetime, timedelta
import datetime
url = 'http://apis.data.go.kr/1360000/WthrWrnInfoService/getWthrWrnList'
params ={'serviceKey' : 'I3k49MWVfMM1ikcVAQtW+aBQeMCQuFa3+ZqXWrCmB1NqsdllN466vryE/9Nt1OhZ3nx46rQ6oaw0nGhO/FJULg==', 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON'}

response = requests.get(url, params=params)
contents = response.text
json_ob = json.loads(contents)
body = json_ob['response']['body']['items']['item']
body = pd.json_normalize(body)
body

,stnId,title,tmFc,tmSeq
0,143,[특보] 제12-79호 : 2022.12.23.11:50 / 강풍주의보 발표 (*),202212231150,79
1,108,[특보] 제12-248호 : 2022.12.23.11:50 / 강풍주의보 발표 (*),202212231150,248
2,156,[특보] 제12-79호 : 2022.12.23.11:00 / 한파주의보 발표 (*),202212231100,79
3,143,[특보] 제12-78호 : 2022.12.23.11:00 / 강풍주의보 변경 (*),202212231100,78
4,133,[특보] 제12-55호 : 2022.12.23.11:00 / 대설주의보 해제 (*),202212231100,55
5,108,[특보] 제12-247호 : 2022.12.23.11:00 / 강풍주의보 변경·대설...,202212231100,247
6,143,[특보] 제12-77호 : 2022.12.23.10:00 / 건조주의보 발표 (*),202212231000,77
7,109,[특보] 제12-59호 : 2022.12.23.10:00 / 풍랑주의보 변경 (*),202212231000,59
8,108,[특보] 제12-246호 : 2022.12.23.10:00 / 풍랑주의보 변경·건조...,202212231000,246
9,159,[특보] 제12-40호 : 2022.12.23.06:00 / 대설주의보 해제 (*),202212230600,40


In [6]:
local_code = pd.read_excel('C://Users//user//Documents//python-streamlit//위기대응 실시간 특보 조회//data//지역코드.xlsx')
local_code = local_code.drop([0],axis = 0)
for i in local_code.index:
    local_code.loc[i,'#STN_ID,']=local_code.loc[i,'#STN_ID,'][:-1]
    local_code.loc[i,'STN_KO,']=local_code.loc[i,'STN_KO,'][:-1]
local_code.rename(columns={'#STN_ID,':'stnId','STN_KO,':'지역명'},inplace=True)
special_report_df = pd.merge(body,local_code)
special_report_df=special_report_df[['stnId', '지역명','title', 'tmFc', 'tmSeq']]
special_report_df['발표시각'] = np.NaN

In [7]:
for i in special_report_df.index:
    year = special_report_df['tmFc'][i].astype('str')[:4] + '년'
    month = special_report_df['tmFc'][i].astype('str')[4:6] + '월'
    day = special_report_df['tmFc'][i].astype('str')[6:8] + '일'
    hour = special_report_df['tmFc'][i].astype('str')[8:10] + '시'
    minute = special_report_df['tmFc'][i].astype('str')[10:12] + '분'
    special_report_df['발표시각'][i] = year + month + day +' '+ hour + minute

special_report_df.rename(columns={'tmSeq': '발표번호'},inplace=True)
special_report_df.rename(columns={'title': '제목'},inplace=True)
special_report_df.rename(columns={'stnId': '지점코드'},inplace=True)
special_report_df.drop(labels='tmFc',axis = 1,inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_1952\3149564083.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special_report_df['발표시각'][i] = year + month + day +' '+ hour + minute


## 특보구역코드

In [10]:
special_local_code = pd.read_excel('C://Users//user//Documents//python-streamlit//위기대응 실시간 특보 조회//data//특보구역코드.xlsx')

for i in local_code.index:
    special_local_code.loc[i,'LAT,']=special_local_code.loc[i,'LAT,'][:-1]
    special_local_code.loc[i,'LON,']=special_local_code.loc[i,'LON,'][:-1]
    special_local_code.loc[i,'#STN_ID,'] = special_local_code.loc[i,'#STN_ID,'][:-1]
    special_local_code.loc[i,'WRN_KO,'] = special_local_code.loc[i,'WRN_KO,'][:-1]
    special_local_code.loc[i,'STN_KO,'] = special_local_code.loc[i,'STN_KO,'][:-1]
special_local_code = special_local_code.drop([0],axis = 0)
special_local_code['LAT,'] = special_local_code['LAT,'].astype('float')
special_local_code['LON,'] = special_local_code['LON,'].astype('float')


special_local_code = special_local_code[['LAT,', 'LON,', 'WRN_KO,', 'STN_KO,','#STN_ID,']]
special_local_code.rename(columns={'#STN_ID,' : '지점코드'},inplace=True)

In [11]:
special_local_code

,"LAT,","LON,","WRN_KO,","STN_KO,",지점코드
1,38.250850,128.56473,속초시평지,속초,90
2,37.947380,127.75443,춘천시,북춘천,93
3,38.147870,127.30420,철원군,철원,95
4,37.239520,131.86983,울릉도독도,독도,96
5,37.901880,127.06070,동두천시,동두천,98
...,...,...,...,...,...
670,35.090040,128.98393,부산광역시,사하,950
671,35.508690,126.90733,정읍시,내장산,951
672,34.998390,128.67977,거제시,장목,953
673,35.431030,129.36011,울산광역시,온산,954


In [12]:
map_lat_lod = pd.merge(special_report_df,special_local_code)
map_lat_lod

,지점코드,지역명,제목,발표번호,발표시각,"LAT,","LON,","WRN_KO,","STN_KO,"
0,143,대구,[특보] 제12-79호 : 2022.12.23.11:50 / 강풍주의보 발표 (*),79,2022년12월23일 11시50분,35.87797,128.65296,대구광역시,대구
1,143,대구,[특보] 제12-78호 : 2022.12.23.11:00 / 강풍주의보 변경 (*),78,2022년12월23일 11시00분,35.87797,128.65296,대구광역시,대구
2,143,대구,[특보] 제12-77호 : 2022.12.23.10:00 / 건조주의보 발표 (*),77,2022년12월23일 10시00분,35.87797,128.65296,대구광역시,대구
3,108,서울,[특보] 제12-248호 : 2022.12.23.11:50 / 강풍주의보 발표 (*),248,2022년12월23일 11시50분,37.57142,126.96580,서울서북권,서울
4,108,서울,[특보] 제12-247호 : 2022.12.23.11:00 / 강풍주의보 변경·대설...,247,2022년12월23일 11시00분,37.57142,126.96580,서울서북권,서울
5,108,서울,[특보] 제12-246호 : 2022.12.23.10:00 / 풍랑주의보 변경·건조...,246,2022년12월23일 10시00분,37.57142,126.96580,서울서북권,서울
6,156,광주,[특보] 제12-79호 : 2022.12.23.11:00 / 한파주의보 발표 (*),79,2022년12월23일 11시00분,35.17294,126.89156,광주광역시,광주
7,133,대전,[특보] 제12-55호 : 2022.12.23.11:00 / 대설주의보 해제 (*),55,2022년12월23일 11시00분,36.37198,127.37211,대전광역시,대전
8,159,부산,[특보] 제12-40호 : 2022.12.23.06:00 / 대설주의보 해제 (*),40,2022년12월23일 06시00분,35.10468,129.03203,부산광역시,부산


## 특보 출력해주기

In [13]:
special_report_df

,지점코드,지역명,제목,발표번호,발표시각
0,143,대구,[특보] 제12-79호 : 2022.12.23.11:50 / 강풍주의보 발표 (*),79,2022년12월23일 11시50분
1,143,대구,[특보] 제12-78호 : 2022.12.23.11:00 / 강풍주의보 변경 (*),78,2022년12월23일 11시00분
2,143,대구,[특보] 제12-77호 : 2022.12.23.10:00 / 건조주의보 발표 (*),77,2022년12월23일 10시00분
3,108,서울,[특보] 제12-248호 : 2022.12.23.11:50 / 강풍주의보 발표 (*),248,2022년12월23일 11시50분
4,108,서울,[특보] 제12-247호 : 2022.12.23.11:00 / 강풍주의보 변경·대설...,247,2022년12월23일 11시00분
5,108,서울,[특보] 제12-246호 : 2022.12.23.10:00 / 풍랑주의보 변경·건조...,246,2022년12월23일 10시00분
6,156,광주,[특보] 제12-79호 : 2022.12.23.11:00 / 한파주의보 발표 (*),79,2022년12월23일 11시00분
7,133,대전,[특보] 제12-55호 : 2022.12.23.11:00 / 대설주의보 해제 (*),55,2022년12월23일 11시00분
8,159,부산,[특보] 제12-40호 : 2022.12.23.06:00 / 대설주의보 해제 (*),40,2022년12월23일 06시00분


In [14]:
import numpy as np
import requests
import pprint
import json
import pandas as pd
import datetime
from datetime import datetime, timedelta
import datetime
import streamlit as st
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MiniMap
from streamlit_folium import st_folium

m = folium.Map(
    location=[special_local_code['LAT,'].mean(), special_local_code['LON,'].mean()],
    zoom_start= 7, width = '70%', height = '50%'
    )
coords = map_lat_lod[['LAT,', 'LON,', 'WRN_KO,', 'STN_KO,']]
text = str(special_report_df['제목']).split('/')
for idx in coords.index:
        result = ''.join(text[idx].split(' ')[1:3])
        folium.Marker([coords.loc[idx, 'LAT,'], coords.loc[idx, 'LON,']], icon = folium.Icon(color="green"), tooltip = coords.loc[idx,'WRN_KO,'] + '<br>[특보] :' + str(result)  ).add_to(m)
m